In [1]:
from bs4 import BeautifulSoup
import csv
import numpy as np
import pandas as pd
from pandas import DataFrame
from collections import Counter
from Bio.SeqUtils import ProtParamData
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

index = ProtParamData.DIWV 

C:\Users\User\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:

def calculate_canonical_value(sequence):
    """
    N: Asparagine, G: Glycine, P: Proline, S: Serine
    R: Arginine, K: Lysine, D: Aspartic acid, E: Glutamic acid
    n: total number of amino acid residues in the protein.
    """
    amino_acids_count = Counter(sequence)
    N = amino_acids_count['N']; G = amino_acids_count['G']; P = amino_acids_count['P']; S = amino_acids_count['S']
    R = amino_acids_count['R']; K = amino_acids_count['K']; D = amino_acids_count['D']; E = amino_acids_count['E']
    n = len(sequence)
    
    canonical_value = 15.43 * ((N+G+P+S)/n) - 29.56 * abs((((R+K)-(D+E))/n-0.03))
    return canonical_value

In [2]:

def calcualte_solubility_index(sequence):
    """
    Calcualte the solubility index
    
    S_TP: Tripeptide Score, AI: Aliphatic Index, II: Instability Index 
    FN: frequency of occurrence of Asparagine (A), FT: frequency of occurrence of Threonine (T)
    FY: frequency of occurrence of Tyrosine (Y)
    D_(ABC,S): 0.2
    
    """
    amino_acids_count = Counter(sequence)
    FN = amino_acids_count['A']; FT = amino_acids_count['T']; FY = amino_acids_count['Y']
    AI = calculate_aliphatic_index(sequence)
    II = calculate_instability_index(sequence)
    S_TP = calculate_tripeptide_score(sequence)
    SI = (0.648 * AI + 0.274 * II - 0.539 * FN - 0.508 * FT - 0.604 * FY - S_TP * 10000)/100
    return SI

def calculate_tripeptide_score(sequence):
    sequence_length = len(sequence)
    sum = 0.0
    for i in range(1, sequence_length):
        sum += 0.2
    return (1/(sequence_length-1)) * sum

def calculate_aliphatic_index(sequence):
    """
    A: Alanine, V: Valine, I: Isoleucine, L: Leucine residues
    """
    a = 2.9
    b = 3.9
    amino_acids_count = Counter(sequence)
    A = amino_acids_count['A']; V = amino_acids_count['V']; I = amino_acids_count['I']; L = amino_acids_count['L']
    AI = A + a * V + b * (I + L)
    return AI

def calculate_instability_index(sequence):
    score = 0.0 
    for i in range(1, len(sequence)-1):
        try:
            dipeptide_value = index[sequence[i]][sequence[i+1]]
        except:
            dipeptide_value = 0.0
        score += dipeptide_value 

    return (10.0 / len(sequence)) * score
#     x = ProteinAnalysis(sequence)
#     return x.instability_index() 
    
    

In [5]:

def file_to_list(path):
    f = open(path)
    soup = BeautifulSoup(f, "html.parser")
    f.close()

    sequence_list = soup.find_all("sequence")
    store_list = list()
    canonical_value_list = list()
    solubility_index_list = list()
#     sequence_list[:5]
    for x in sequence_list:
        if x.contents and x.contents != '':
            sequence_string = (''.join(str(e) for e in x.contents)).replace('\n', '').strip()
            store_list.append(sequence_string)
            canonical_value_list.append(calculate_canonical_value(sequence_string))
            solubility_index_list.append(calcualte_solubility_index(sequence_string))
    return store_list, canonical_value_list, solubility_index_list

soluble_path = r'C:\Users\User\Downloads\Soluble Subset.xhtml'
insoluble_path = r'C:\Users\User\Downloads\Insoluble Subset.xhtml'

soluble_list, soluble_canonical_list, soluble_solubility_index_list = file_to_list(soluble_path)
insoluble_list, insoluble_canonical_list, insoluble_solubility_index_list  = file_to_list(insoluble_path)
print(len(soluble_list)+len(insoluble_list))


2692


In [38]:
df1 = DataFrame(
    {
        'Canonical Value': soluble_canonical_list,
        'Solubility Index': soluble_solubility_index_list,
        'Label': 1,
        'Sequence': soluble_list
        
    }
)
# df1.reindex(columns=["Label","Canonical ","Solubility Index","Sequence"]) 
# df1 = DataFrame([[1],soluble_canonical_list,soluble_solubility_index_list,soluble_list],
#                columns=["Label","Canonical Value","Solubility Index","Sequence"]
#                )
# df2 = DataFrame([[0],insoluble_canonical_list,insoluble_solubility_index_list,insoluble_list],
#                columns=["Label","Canonical Value","Solubility Index","Sequence"]
#                )
df2 = DataFrame(
    {
        'Canonical Value': insoluble_canonical_list,
        'Solubility Index': insoluble_solubility_index_list,
        'Label': 0,
        'Sequence': insoluble_list
        
    }
)
# df1.reindex(columns=["Label","Canonical Value","Solubility Index","Sequence"])
frames = [df1, df2]
# frames = frames[['Label', 'Sequence', 'Solubility Index', 'Canonical Value']]
concatenated_df = pd.concat(frames)
concatenated_df.to_csv(r'C:\Users\User\complete_dataset.csv', sep=',', index=False)
# writer.save()
# df['Label'] = 1
df = concatenated_df.dropna(axis=0, how='any')
# df.to_excel(r'C:\Users\User\complete_dataset.xlsx', sheet_name='sheet1', index=False)
# len(df)
# with open('C:\Users\User\complete_dataset.xls', "w") as csv_file:
#         writer = csv.writer(csv_file)
#         writer.writerow(store_list)



In [51]:
df = pd.read_csv(r'complete_dataset.csv')
df = df.dropna(axis=0, how='all')
print(df.head())

   Canonical Value  Label                                           Sequence  \
0         2.032131      1  MKTPWKVLLGLLGVAALVTIITVPVVLLNKDEAAADSRRTYTLADY...   
1         1.464755      1  MKQSPALAPEERCRRAGSPKPVLRADDNNMGNGCSQKLATANLLRF...   
2         2.471316      1  MFCTKLKDLKITGECPFSLLAPGQVPNESSEEAAGSSESCKATVPI...   
3         2.706376      1  MGWLCSGLLFPVSCLVLLQVASSGNMKVLQEPTCVSDYMSISTCEW...   
4         2.936336      1  MGRLCTKFLTSVGCLILLLVTGSGSIKVLGEPTCFSDYIRTSTCEW...   

   Solubility Index  
0        -16.727891  
1        -15.724514  
2        -16.495141  
3        -16.543848  
4        -16.427135  


In [52]:
print(df.shape)

(2692, 4)


In [54]:
from sklearn.preprocessing import LabelEncoder

x = df.copy()
for column in x.columns:
    if x[column].dtype == type(object):
        le = LabelEncoder()
        le.fit(x[column])
        x[column] = le.transform(x[column])

print(x.head(5))
print(df['Sequence'][:2])
Y = x.values[:, 1] # this consists of the outcome variable.
del x['Label']
X = x.values[:, :] # this consists of predictor variables.
# df['Label'].dtype
# x['Sequence']
X_train , X_test, y_train, y_test =train_test_split(X, Y, test_size=0.3, random_state=100)

# X_train.shape




   Canonical Value  Label  Sequence  Solubility Index
0         2.032131      1       805        -16.727891
1         1.464755      1       774        -15.724514
2         2.471316      1       562        -16.495141
3         2.706376      1       653        -16.543848
4         2.936336      1       636        -16.427135
0    MKTPWKVLLGLLGVAALVTIITVPVVLLNKDEAAADSRRTYTLADY...
1    MKQSPALAPEERCRRAGSPKPVLRADDNNMGNGCSQKLATANLLRF...
Name: Sequence, dtype: object


In [55]:
clf_gini = DecisionTreeClassifier(criterion="entropy", random_state=100, max_depth=2 , min_samples_leaf=5)
clf_gini.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [65]:
s = "MKTPWKVLLGLLGVAALVTIITVPVVLLNKDEAAADSRRTYTLADYLKNTFRVKSYSLRWVSDSEYLYKQENNILLFNAEHGNSSIFLENSTFEIFGDSISDYSVSPDRLFVLLEYNYVKQWRHSYTASYSIYDLNKRQLITEEKIPNNTQWITWSQEGHKLAYVWKNDIYVKIEPHLPSHRITSTGKENVIFNGINDWVYEEEIFGAYSALWWSPNGTFLAYAQFNDTGVPLIEYSFYSDESLQYPKTVWIPYPKAGAVNPTVKFFIVNTDSLSSTTTTIPMQITAPASVTTGDHYLCDVAWVSEDRISLQWLRRIQNYSVMAICDYDKTTLVWNCPTTQEHIETSATGWCGRFRPAEPHFTSDGSSFYKIVSDKDGYKHICQFQKDRKPEQVCTFITKGAWEVISIEALTSDYLYYISNEYKEMPGGRNLYKIQLTDHTNKKCLSCDLNPERCQYYSVSLSKEAKYYQLGCRGPGLPLYTLHRSTDQKELRVLEDNSALDKMLQDVQMPSKKLDFIVLNETRFWYQMILPPHFDKSKKYPLLIDVYAGPCSQKADAAFRLNWATYLASTENIIVASFDGRGSGYQGDKIMHAINKRLGTLEVEDQIEAARQFLKMGFVDSKRVAIWGWSYGGYVTSMVLGSGSGVFKCGIAVAPVSRWEYYDSVYTERYMGLPTPEDNLDHYRNSTVMSRAENFKQVEYLLIHGTADDNVHFQQSAQISKALVDAGVDFQAMWYTDEDHGIASSTAHQHIYSHMSHFLQQCFSLR"
le = LabelEncoder()
s = np.array([s])
le.fit(s)
ss = le.transform(s)
print(ss)
# clf_gini.predict([[-2.02984348, 843, ss]]) # predict target variable for test set’s 1st record.

[1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0]


In [49]:
y_pred = clf_gini.predict(X_test)
y_pred[15]
X_test[15]

array([ -2.72984348,  34.        , -19.88289791])

In [50]:
print("Accuracy is ", accuracy_score(y_test,y_pred)*100)

Accuracy is  62.5
